In [1]:
# script to determine number of moves in a game

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import ast

In [3]:
#filename = "3x3game_p1minimax0_p2minimax5_2022_04_11-07_54_04_PM.csv"
#filename = "output_5x5.csv"
#
#df = pd.read_csv(filename, header=None)
#df.columns = ["Previous State", "Next State", "Player", "Cyber Move", "Game ID"]

In [4]:
#df

In [5]:
def get_nmoves(df, game_id):
    value_counts = df["Game ID"].value_counts()
    return value_counts[game_id]

In [8]:
def average_nmoves(filename):
    
    df = pd.read_csv(filename, header=None)
    df.columns = ["Previous State", "Next State", "Player", "Cyber Move", "Game ID"]

    moves = 0

    for each in np.unique(df["Game ID"]):
        nmoves = get_nmoves(df, game_id=each)
        moves += nmoves

    n = moves/len(np.unique(df["Game ID"]))

    print(f"Average number of moves per game in {filename}: {n:4.2f}")
    
    return n

In [10]:
average_nmoves(filename="3x3game_p1minimax0_p2minimax5_2022_04_11-07_54_04_PM.csv")

Average number of moves per game in 3x3game_p1minimax0_p2minimax5_2022_04_11-07_54_04_PM.csv: 6.22


6.22